In [1]:
import pandas as pd
import psycopg2
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Carregando o Dataset
df = pd.read_pickle('../storage/pickle/dataframe_amostra_no_dummies0.15.pkl')
df = df.drop_duplicates()
# Tratando dado de idade
df["idade"] = pd.to_numeric(df["idade"])

KeyboardInterrupt: 

In [3]:
# Conecta ao banco de dados
conn = psycopg2.connect(
    host="localhost",
    database="previsao-patologias",
    user="postgres",
    password="test"
)

cursor = conn.cursor()
query = "SELECT texto FROM tb_questoes WHERE tipo_resposta = 'binaria';"
cursor.execute(query)

# Pega resultado da query e transforma em lista para realizar iteração
tuplas_resultados = cursor.fetchall()
questoes = [item for tupla in tuplas_resultados for item in tupla]

In [4]:
questoes_binarias = []
for c in questoes:
    nome_coluna_tratado = c.replace(" ", "_").replace(".", "").replace("?", "").lower()
    questoes_binarias.append(nome_coluna_tratado)
    
colunas_dummies = [col for col in df.columns if col not in ['idade', 'patologia', 'severidade'] + questoes_binarias]

df_dummies = pd.get_dummies(df, columns=colunas_dummies, dtype=int)

Separando variáveis dependentes e de testes e criando conjunto de treino e testes

In [5]:
variaveis_dependentes = df_dummies.drop(columns=['patologia', 'severidade'])
variavel_target = df_dummies['patologia']

In [6]:
# Splitando conjuntos de treino e testes
X_train, X_test, y_train, y_test = train_test_split(variaveis_dependentes, variavel_target, test_size=0.2, random_state=42)

In [7]:
rna = MLPClassifier(
    hidden_layer_sizes=(5,),
    random_state=42,
    max_iter=10,
    activation="relu"
)


Realizando Fit das Variáveis e Predição das Observações

In [8]:
# Realizando o fit das variáveis
rna.fit(X_train, y_train)

# Fazer previsões
y_pred = rna.predict(X_test)

/home/guilhermehip/.pyenv/versions/3.10.13/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


In [9]:
# Obtenha informações sobre a arquitetura da MLP
architecture = [X_train.shape[1]] + list(rna.hidden_layer_sizes) + [len(set(y_train))]
architecture

[610, 5, 49]

Utilizando técnica de acurácia comum

In [10]:
# Avaliar o desempenho do modelo
accuracy = accuracy_score(y_test, y_pred)
classification_report_result = classification_report(y_test, y_pred)
matriz_confusao = confusion_matrix(y_test, y_pred, labels=rna.classes_)
f1 = f1_score(y_test, y_pred, average='macro')

# Exibir métricas de desempenho
print(f'Acurácia: {accuracy}')
print('Relatório de Classificação:\n', classification_report_result)
print(f"F1-Score: {f1}")

Acurácia: 0.9927427685950413
Relatório de Classificação:
                                           precision    recall  f1-score   support

     acute copd exacerbation / infection       1.00      1.00      1.00       660
                acute dystonic reactions       1.00      1.00      1.00      1017
                        acute laryngitis       0.99      0.97      0.98       905
                      acute otitis media       1.00      1.00      1.00       978
                   acute pulmonary edema       0.99      1.00      1.00       699
                    acute rhinosinusitis       0.99      0.77      0.87       487
                      allergic sinusitis       1.00      1.00      1.00       879
                             anaphylaxis       1.00      1.00      1.00      1056
                                  anemia       1.00      1.00      1.00      1939
                     atrial fibrillation       1.00      1.00      1.00       807
                               boerhaav

Utilizando K-fold Cross Validation

In [15]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
acuaria_validacao_cruzada = cross_val_score(rna, variaveis_dependentes, variavel_target, cv=kfold, scoring='f1_macro')

# Exibir a acurácia média e o desvio padrão
print(f'F1-Score Médio (k-fold): {acuaria_validacao_cruzada.mean()}')
print(f'Desvio Padrão da Acurácia (k-fold): {acuaria_validacao_cruzada.std()}')

/home/guilhermehip/.pyenv/versions/3.10.13/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


In [3]:
# Carregando o Dataset
df = pd.read_pickle('../storage/pickle/dataframe_smote0.10.pkl')
df = df.drop_duplicates()
# Tratando dado de idade
df["idade"] = pd.to_numeric(df["idade"])

In [4]:
variaveis_dependentes = df.drop(columns=['patologia', 'severidade'])
variavel_target = df['patologia']

In [5]:
# Splitando conjuntos de treino e testes
X_train, X_test, y_train, y_test = train_test_split(variaveis_dependentes, variavel_target, test_size=0.2, random_state=42)

In [11]:
rna = MLPClassifier(
    hidden_layer_sizes=(5,),
    random_state=42,
    max_iter=10,
    activation="relu"
)


In [12]:
# Realizando o fit das variáveis
rna.fit(X_train, y_train)

# Fazer previsões
y_pred = rna.predict(X_test)

/home/guilhermehip/.pyenv/versions/3.10.13/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


In [13]:
# Avaliar o desempenho do modelo
accuracy = accuracy_score(y_test, y_pred)
classification_report_result = classification_report(y_test, y_pred)
matriz_confusao = confusion_matrix(y_test, y_pred, labels=rna.classes_)
f1 = f1_score(y_test, y_pred, average='macro')

# Calcular sensibilidade (recall) e especificidade para cada classe
sensibilidade = np.diag(matriz_confusao) / np.sum(matriz_confusao, axis=1)

# Calcular sensibilidade e especificidade médias (macro)
sensibilidade_media = np.mean(sensibilidade)

# Exibir métricas de desempenho
print(f'Acurácia: {accuracy}')
print('Relatório de Classificação:\n', classification_report_result)
print(f"F1-Score: {f1}")
print(f'Sensibilidade (Recall) Média: {sensibilidade_media}')

Acurácia: 0.9925245406493833
Relatório de Classificação:
                                           precision    recall  f1-score   support

     acute copd exacerbation / infection       0.99      1.00      1.00       803
                acute dystonic reactions       1.00      1.00      1.00       781
                        acute laryngitis       0.99      0.99      0.99       829
                      acute otitis media       1.00      0.99      1.00       850
                   acute pulmonary edema       1.00      1.00      1.00       796
                    acute rhinosinusitis       0.93      0.82      0.87       820
                      allergic sinusitis       1.00      0.99      1.00       789
                             anaphylaxis       1.00      1.00      1.00       851
                                  anemia       1.00      1.00      1.00       775
                     atrial fibrillation       1.00      1.00      1.00       797
                               boerhaav

: 